In [1]:
import tweepy
import pandas as pd
import gmaps
import googlemaps
from tqdm import tqdm

In [2]:
consumer_key=""
consumer_secret=""
access_token=""
access_token_secret=""

In [3]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)

In [16]:
tweets = tweepy.Cursor(api.search, q="flu -filter:retweets", lang="en", result_type="recent").items()

In [17]:
tweets_list = []
for tweet in tweets:
    tweets_list.append([tweet.id, tweet.coordinates, tweet.user.location])

In [18]:
df = pd.DataFrame(data=tweets_list, 
                  columns=['id', "coordinates", "location"])

In [19]:
df

,id,coordinates,location
0,1190873754814926848,None,"Hacienda Heights, CA"
1,1190873452204437505,None,"The Copper Beeches, UK"
2,1190873402195595264,None,
3,1190873210675453952,None,Washington D.C. area
4,1190872907129462784,None,"Wisconsin, USA"
...,...,...,...
65162,1187663410638573574,None,
65163,1187663396835119104,None,
65164,1187662848073158658,None,"Washington, USA"
65165,1187662215937171456,None,The Borders


In [20]:
gmaps_client = googlemaps.Client(key="")

In [21]:
def in_usa(address_components):
    for component in address_components:
        if "country" in component["types"]:
            short_lower = component["short_name"].lower()
            if short_lower in ["us", "usa"]:
                return True
    return False

In [22]:
coordinates = []
for i in tqdm(range(len(df.location))):
    x = df.location[i]
    if not x:
        continue
    geocode_result = gmaps_client.geocode(x)
    if geocode_result:
        address_components = geocode_result[0]["address_components"]
        usa_location = in_usa(address_components)
        if usa_location:
            lat = geocode_result[0]["geometry"]["location"]["lat"]
            lon = geocode_result[0]["geometry"]["location"]["lng"]
            coordinates.append([lat, lon])

100%|██████████████████████████████████████████████████████████████████████████| 65167/65167 [3:27:45<00:00,  5.23it/s]


In [23]:
print(len(coordinates))

26795


In [24]:
coordinates = pd.DataFrame(coordinates, columns=["latitude", "longitude"])

In [3]:
coordinates = pd.read_csv("coordinates.csv")[["latitude", "longitude"]]

In [4]:
gmaps.configure(api_key="") # Fill in with your API key

In [6]:
heatmap_layer = gmaps.heatmap_layer(coordinates)
heatmap_layer.max_intensity = 100
heatmap_layer.point_radius = 10

figure_layout = {
    'width': '1000px',
    'height': '1000px'
}
fig = gmaps.figure(map_type="HYBRID", layout=figure_layout, center=[37.0902, -110.7129], zoom_level=3.6)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='1000px', width='1000px'))

In [26]:
coordinates.to_csv("coordinates.csv")

In [ ]:
print("Checkpoint 1")
scatter = gmaps.symbol_layer(
    coordinates, fill_color='red', stroke_color='green', scale=2
)

figure_layout = {
    'width': '1000px',
    'height': '1000px'
}

print("Checkpoint 2")
fig = gmaps.figure(layout=figure_layout, center=[37.0902, -110.7129], zoom_level=3.6)
print("Checkpoint 3")
fig.add_layer(scatter)
fig

Checkpoint 1
